In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import ast
import csv

def convert_dataset(dataset):
  examples = []

  for dat, trc in zip(dataset['txData'], dataset['txTrace']):  
    data = ast.literal_eval(dat)
    trace = ast.literal_eval(trc)
    examples.append([
      int(data.get('blockNumber'), 0),
      int(data.get('from'), 0) % (2 ** 30),
      (int(data.get('to'), 0) if data.get('to') is not None else 0) % (2 ** 30),
      int(data.get('gas'), 0),
      ((int(trace.get('gasUsed'), 0) if trace.get('gasUsed') is not None else 0) *int(data.get('gasPrice'), 0)),
      (int(data.get('input')[:10], 0) if data.get('input') != '0x' else 0) % (2 ** 30),
      int(data.get('nonce'), 0),
      (1 if  trace.get('error') is not None else 0)
    ])
  return np.array(examples)

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
testFeatures = convert_dataset(test)
 
binaryModel = xgb.XGBClassifier(n_estimators=50)
binaryModel.fit(convert_dataset(train), train['Label0'])
binaryPredictions = binaryModel.predict_proba(testFeatures)[:, 1]
 
regressionModel = xgb.XGBRegressor(n_estimators=50)
regressionModel.fit(
  convert_dataset(train[train['Label0'] == True]),
  train[train['Label0'] == True]['Label1']
)
regressionPredictions = regressionModel.predict(testFeatures)
 
submission = csv.writer(open('submission.csv', 'w', encoding='UTF8'))
for x, y in zip(binaryPredictions, regressionPredictions):
  submission.writerow([x, y])  
